In [2]:
import pandas as pd
import os

def writeData(filepath, has_header=True):
    if has_header:
        return pd.read_csv(filepath, header=0)  # First file: use header
    else:
        return pd.read_csv(filepath, header=None)  # Others: treat first row as data

def clearDirtyData(df):
    # Ensure we're comparing actual values, not strings
    df = df.replace([float('inf'), -float('inf')], 'Infinity')
    dropList = df[(df.iloc[:, 14] == "Nan") | (df.iloc[:, 15] == "Infinity")].index.tolist()
    return dropList

def mergeData():
    base_path = r'C:\Users\bzz00\Downloads\archive'
    files = [
        r"Monday-WorkingHours.pcap_ISCX.csv",
        "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
        "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
        "Friday-WorkingHours-Morning.pcap_ISCX.csv",
        "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
        "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
        "Tuesday-WorkingHours.pcap_ISCX.csv",
        "Wednesday-workingHours.pcap_ISCX.csv"
    ]

    # Load and store all DataFrames
    data_frames = [writeData(os.path.join(base_path, f)) for f in files]

    # Merge all DataFrames
    result = pd.concat(data_frames, ignore_index=True)

    # Clean dirty data
    dirty_indices = clearDirtyData(result)
    result = result.drop(dirty_indices)

    return result

# Load merged and cleaned data
raw_data = mergeData()
raw_data


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
# 得到标签列索引
# Count the number of good and bad flows
label_counts = raw_data[' Label'].value_counts()
print(label_counts)

BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name:  Label, dtype: int64


In [8]:
# Simplify to Good (BENIGN) vs Bad (everything else)
raw_data['FlowType'] = raw_data[' Label'].apply(lambda x: 'Good' if x == 'BENIGN' else 'Bad')
flow_type_counts = raw_data['FlowType'].value_counts()
print(flow_type_counts)

Good    2271320
Bad      556556
Name: FlowType, dtype: int64


### Undersample the majority (BENIGN)

In [9]:
from sklearn.utils import resample

# Separate classes
df_good = raw_data[raw_data['FlowType'] == 'Good']
df_bad = raw_data[raw_data['FlowType'] == 'Bad']

# Downsample good flows
df_good_downsampled = resample(df_good, replace=False, n_samples=len(df_bad), random_state=42)

# Combine back
df_balanced = pd.concat([df_good_downsampled, df_bad])


In [10]:
df_balanced['FlowType'].value_counts()

Good    556556
Bad     556556
Name: FlowType, dtype: int64

In [12]:
df_balanced.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,FlowType
1522001,53,206,2,2,68,132,34,34,34.000000,0.000000,...,0.0000,0.0000,0,0,0.0,0.000000,0,0,BENIGN,Good
806072,80,116803674,20,16,1368,5895,424,0,68.400000,153.268392,...,104666.7273,260303.8732,889386,22878,10000000.0,577.998978,10000000,9999141,BENIGN,Good
213674,53,48698,2,2,88,192,44,44,44.000000,0.000000,...,0.0000,0.0000,0,0,0.0,0.000000,0,0,BENIGN,Good
1676447,53,79283,1,1,56,137,56,56,56.000000,0.000000,...,0.0000,0.0000,0,0,0.0,0.000000,0,0,BENIGN,Good
1894285,443,294065,19,29,871,39643,389,0,45.842105,105.687308,...,0.0000,0.0000,0,0,0.0,0.000000,0,0,BENIGN,Good


## Data Preparation Pipeline

✅ Key Selection Criteria
Keep features related to:

Traffic volume and duration

Inter-arrival times

Flag counts (e.g., SYN, FIN, RST)

Packet length stats

Flow-based behavior (bulk rate, segment size)

Init window size (indicates TCP behavior)

Anything tied to burstiness, anomalies, or TCP irregularities

Remove features that are:

Mostly constant or irrelevant (e.g., Destination Port)

Redundant (e.g., duplicate header length columns)

Too sparse or specific to certain protocols (e.g., bulk rate fields often zero)

In [13]:
df_balanced.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

🗑️ Remove/Skip These:
' Destination Port' – non-informative for general models

'Fwd Avg Bytes/Bulk', 'Bwd Avg Bulk Rate', etc. – often sparse (zero-filled)

' Fwd Header Length.1' – duplicate of ' Fwd Header Length'

'Flow Bytes/s', ' Flow Packets/s' – derived and sometimes error-prone

'Idle Min', 'Active Min' – generally less informative than mean/std/max

In [15]:
df_balanced.columns = df_balanced.columns.str.strip()
selected_features = [
    "Flow Duration",
    "Total Fwd Packets",
    "Total Backward Packets",
    "Total Length of Fwd Packets",
    "Total Length of Bwd Packets",
    "Fwd Packet Length Max",
    "Fwd Packet Length Mean",
    "Bwd Packet Length Max",
    "Bwd Packet Length Mean",
    "Flow IAT Mean",
    "Fwd IAT Mean",
    "Bwd IAT Mean",
    "Fwd PSH Flags",
    "Bwd PSH Flags",
    "Fwd Header Length",
    "Bwd Header Length",
    "Min Packet Length",
    "Max Packet Length",
    "Packet Length Mean",
    "FIN Flag Count",
    "SYN Flag Count",
    "RST Flag Count",
    "PSH Flag Count",
    "ACK Flag Count",
    "Down/Up Ratio",
    "Average Packet Size",
    "Avg Fwd Segment Size",
    "Avg Bwd Segment Size",
    "Subflow Fwd Packets",
    "Subflow Fwd Bytes",
    "Subflow Bwd Packets",
    "Subflow Bwd Bytes",
    "Init_Win_bytes_forward",
    "Init_Win_bytes_backward",
    "act_data_pkt_fwd",
    "Active Mean",
    "Idle Mean",
    "Label",
    "FlowType"
]

# Filter the dataset
df_filtered = df_balanced[selected_features]

In [22]:
pd.set_option('display.max_columns', None)

df_filtered.head(-20)


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Flow IAT Mean,Fwd IAT Mean,Bwd IAT Mean,Fwd PSH Flags,Bwd PSH Flags,Fwd Header Length,Bwd Header Length,Min Packet Length,Max Packet Length,Packet Length Mean,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,Active Mean,Idle Mean,Label,FlowType
1522001,206,2,2,68,132,34,34.000000,66,66.000000,6.866667e+01,3.000000e+00,3.00,0,0,64,64,34,66,46.800000,0,0,0,0,0,1,58.500000,34.000000,66.000000,2,68,2,132,-1,-1,1,0.0000,0.0,BENIGN,Good
806072,116803674,20,16,1368,5895,424,68.400000,1963,368.437500,3.337248e+06,6.147562e+06,7409620.40,0,0,412,464,0,1963,196.297297,0,0,0,1,0,0,201.750000,68.400000,368.437500,20,1368,16,5895,8192,1013,19,104666.7273,10000000.0,BENIGN,Good
213674,48698,2,2,88,192,44,44.000000,96,96.000000,1.623267e+04,3.000000e+00,4.00,0,0,40,40,44,96,64.800000,0,0,0,0,0,1,81.000000,44.000000,96.000000,2,88,2,192,-1,-1,1,0.0000,0.0,BENIGN,Good
1676447,79283,1,1,56,137,56,56.000000,137,137.000000,7.928300e+04,0.000000e+00,0.00,0,0,40,20,56,137,83.000000,0,0,0,0,0,1,124.500000,56.000000,137.000000,1,56,1,137,-1,-1,0,0.0000,0.0,BENIGN,Good
1894285,294065,19,29,871,39643,389,45.842105,4344,1367.000000,6.256702e+03,1.633694e+04,7503.50,0,0,620,936,0,4344,826.816326,0,0,0,1,0,1,844.041667,45.842105,1367.000000,19,871,29,39643,65535,122,6,0.0000,0.0,BENIGN,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830233,11504291,5,5,348,3525,348,69.600000,2077,705.000000,1.278255e+06,1.626580e+06,2876054.00,0,0,168,168,0,2077,352.090909,0,0,0,1,0,1,387.300000,69.600000,705.000000,5,348,5,3525,29200,235,1,922.0000,6500661.0,DoS GoldenEye,Bad
2830248,11508795,8,5,382,11632,382,47.750000,10184,2326.400000,9.590662e+05,9.295539e+05,2877196.00,0,0,264,168,0,10184,858.142857,0,0,0,1,0,0,924.153846,47.750000,2326.400000,8,382,5,11632,29200,235,1,887.0000,6502817.0,DoS GoldenEye,Bad
2830298,11507323,8,6,610,11632,610,76.250000,5840,1938.666667,8.851787e+05,9.295073e+05,2301455.40,0,0,264,200,0,5840,816.133333,0,0,0,1,0,0,874.428571,76.250000,1938.666667,8,610,6,11632,29200,236,1,821.0000,6501660.0,DoS GoldenEye,Bad
2830316,11507815,7,5,403,11632,403,57.571429,10184,2326.400000,1.046165e+06,1.084686e+06,2876949.50,0,0,232,168,0,10184,925.769231,0,0,0,1,0,0,1002.916667,57.571429,2326.400000,7,403,5,11632,29200,235,1,870.0000,6501855.0,DoS GoldenEye,Bad


In [25]:
cols_to_drop = [
    'FIN Flag Count',
    'SYN Flag Count',
    'RST Flag Count',
    'PSH Flag Count',
    'ACK Flag Count',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Down/Up Ratio',
    'act_data_pkt_fwd'
]

# Drop the columns
df_cleaned =df_filtered.drop(columns=cols_to_drop).reset_index(drop=True)

# Optional: Check remaining columns
df_cleaned

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Flow IAT Mean,Fwd IAT Mean,Bwd IAT Mean,Fwd Header Length,Bwd Header Length,Min Packet Length,Max Packet Length,Packet Length Mean,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Mean,Idle Mean,Label,FlowType
0,206,2,2,68,132,34,34.000000,66,66.000000,6.866667e+01,3.000000e+00,3.0,64,64,34,66,46.800000,58.500000,34.000000,66.000000,2,68,2,132,-1,-1,0.0000,0.0,BENIGN,Good
1,116803674,20,16,1368,5895,424,68.400000,1963,368.437500,3.337248e+06,6.147562e+06,7409620.4,412,464,0,1963,196.297297,201.750000,68.400000,368.437500,20,1368,16,5895,8192,1013,104666.7273,10000000.0,BENIGN,Good
2,48698,2,2,88,192,44,44.000000,96,96.000000,1.623267e+04,3.000000e+00,4.0,40,40,44,96,64.800000,81.000000,44.000000,96.000000,2,88,2,192,-1,-1,0.0000,0.0,BENIGN,Good
3,79283,1,1,56,137,56,56.000000,137,137.000000,7.928300e+04,0.000000e+00,0.0,40,20,56,137,83.000000,124.500000,56.000000,137.000000,1,56,1,137,-1,-1,0.0000,0.0,BENIGN,Good
4,294065,19,29,871,39643,389,45.842105,4344,1367.000000,6.256702e+03,1.633694e+04,7503.5,620,936,0,4344,826.816326,844.041667,45.842105,1367.000000,19,871,29,39643,65535,122,0.0000,0.0,BENIGN,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113107,11512204,8,5,326,11632,326,40.750000,10184,2326.400000,9.593503e+05,9.301100e+05,2878039.5,264,168,0,10184,854.142857,919.846154,40.750000,2326.400000,8,326,5,11632,29200,235,892.0000,6507197.0,DoS GoldenEye,Bad
1113108,11513325,5,5,471,3525,471,94.200000,2077,705.000000,1.279258e+06,1.628158e+06,2878318.0,168,168,0,2077,363.272727,399.600000,94.200000,705.000000,5,471,5,3525,29200,235,918.0000,6508582.0,DoS GoldenEye,Bad
1113109,11509201,7,6,314,11632,314,44.857143,5792,1938.666667,9.591001e+05,1.084509e+06,2301830.6,232,200,0,5792,853.285714,918.923077,44.857143,1938.666667,7,314,6,11632,29200,235,899.0000,6503248.0,DoS GoldenEye,Bad
1113110,11509095,8,5,369,11632,369,46.125000,10184,2326.400000,9.590912e+05,9.296849e+05,2877262.0,264,168,0,10184,857.214286,923.153846,46.125000,2326.400000,8,369,5,11632,29200,235,914.0000,6504954.0,DoS GoldenEye,Bad
